In [9]:
import csv
import concurrent
import json
import requests


skuSetName = "savoy"
skus = json.load(open(f"{skuSetName}Sku.json"))

userAgent = "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0"
"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",

# Add Coles Subscription Key
subscriptionKey = ""

# Add Coles Refresh Token (Optional Store API)
refreshToken = ""

# apiVersion needs to be updated before running the APIs. Coles updates it daily, and depreciates old APIs 
apiVersion = "20240527.02_v3.91.0"

In [ ]:
# Unused code, for the get stores in within x distance APIs
# Abandoned it once we realised the distance flag had a limit. 
# In theory we can get more stores from this, but it's not really worth it.

# 1 - Coles Express
# 2 - Coles Supermarket
# 3 - Liquorland
# 4 - 1st Choice
# 5 - Vintage Cellars
# 6 ... 12 - Not Assigned

brand = ["","Coles Express", "Coles Supermarkets", "Liquor Land", "First Choice", "Vintage Sellers", "6", "7", "8", "9", "10", "11", "12"]
include = [False, True, True, False, False, False, True, True, True, True, True, True, True]
brands = ""

for x in range(len(brand)):
    if include[x] == True:
        brands += f"{x}%2C"
brands = brands[:-3]

numberStores = 50
distance = 5000
lat = lon = 0
        
# Location Search URL (Product and Find-Stores Page):
"https://www.coles.com.au/api/bff/locations/search/suggestions?limit=6&searchTerm=traralgo"

# Product Store Search URL:
"https://www.coles.com.au/api/bff/locations/search?latitude=-38.193322&longitude=146.538373&distance=50&numberOfLocations=20"

# Find-Stores Store Search URL:
"https://www.coles.com.au/api/bff/stores/search?includes=nextDoorStores&latitude=-38.193322&longitude=146.538373&brandIds=1%2C2%2C3%2C4%2C5&numberOfStores=50&distance=5000"

x = requests.get('https://www.coles.com.au/api/bff/stores/search?' +
                 # Shows the stores attached to this store (Eg Liquor store)
                # 'includes=nextDoorStores&' +
                 # latitude=-38.193322
                f'latitude={lat}&' +
                 # longitude=146.538373
                f'longitude={lon}&' + 
                 # brandIds=1%2C2%2C3%2C4%2C5
                f'brandIds={brands}&' +
                 # numberOfStores=50
                 f'numberOfStores={numberStores}&' +
                 # distance=5000
                 f'distance={distance}', 
                 headers = {
                    "ocp-apim-subscription-key": subscriptionKey, 
                 },
                # verify=False
                )
# print(x.json())

In [10]:
### This function calls the inventory API to get athe stock status for a store and an item.
stockUrl1 = f"https://www.coles.com.au/_next/data/{apiVersion}/en/product/"

def stockCheck1(storeId, skuId, url=stockUrl1):
    if url[len(url) -1] != "/":
        url[len(url)] = "/"
    url = f'{url}{skus[skuId]["name"]}-{skuId}.json'
    try:
        stockLevel = requests.get(url, cookies={"fulfillmentStoreId": str(storeId)}, headers={"User-Agent": userAgent}, verify=False)
    except Exception as e:
        print(f"ERR - {storeId} - {skuId} - {e}")
        return "Error"

    store = {}
    store["id"] = storeId

    if stockLevel.status_code != 200 and stockLevel.status_code != 404:
        print(f"WARN {storeId} - {skuId} - {stockLevel.status_code} - {stockLevel.text}")
        return "Error"

    else:
        try:
            stockLevel = stockLevel.json()
            if "notFound" in stockLevel:
                return "Not Found"
            elif "pageProps" in stockLevel and "product" in stockLevel["pageProps"]:
                if "pricing" in stockLevel["pageProps"]["product"] and stockLevel["pageProps"]["product"]["pricing"] is None:
                    return "Not Stocked"
                elif stockLevel["pageProps"]["product"]["availability"]:
                    return "In Stock"
                else:
                    return "Out of Stock"
            else:
                print(f"ERR - {storeId} - {skuId} - Misformed JSON -{stockLevel}")
                return "Error"
        except:
            print(f"ERR - {storeId} - {url} - {stockLevel.text}")
            return "Error"

In [ ]:
ids = range(100,10000)
# ids = [1284, 694, 684, 696]
storeResults = {}
storeThreadResults = {}
skuId = list(skus.keys())[0]
storesNotColesOnline = []
storeErrors = {}
storeErrors[skuId] = []

with concurrent.futures.ThreadPoolExecutor() as executor: # optimally defined number of threads
    for id in ids:
        storeThreadResults[id] = executor.submit(stockCheck1, str(id), skuId)
    
for id, store in storeThreadResults.items():
    tempResult = store.result()
    if tempResult == "Error":
        storeErrors["initialSku"].append(id)
        print(f"ERR - {id}")
    elif tempResult != "Not Found":
        storeResults[id] = {}
        storeResults[id][skuId] = tempResult
    else:
        storesNotColesOnline.append(id)
        print(f"Warn - {id}")

print(f"Not Online - {storesNotColesOnline}")
print(f'Error - {storeErrors[skuId]}')

json_output = json.dumps(storeResults)
with open(f'{skuSetName}Coles1.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storesNotColesOnline)
with open(f'{skuSetName}ColesNotOnline.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storeErrors)
with open(f'{skuSetName}ColesError.json', "w") as f:
    f.write(json_output)

In [ ]:
#This query the other 3 skus, to get a complete picture of which stores are selling what

storeResults = json.load(open(f'{skuSetName}Coles1.json'))
storeErrors = json.load(open(f'{skuSetName}ColesError.json'))
skuId = list(list(storeResults.values())[0].keys())[0]
ids = list(storeResults.keys())
remainingSkus = []

for key in skus.keys():
    if key != skuId:
        remainingSkus.append(key)
        storeErrors[key] = []

storeErrors["notOnline"] = []
storeNotSelling = []

storeThreadResults = {}

with concurrent.futures.ThreadPoolExecutor() as executor: # optimally defined number of threads
    for id in ids:
        storeThreadResults[id] = {}
        for skuId2 in remainingSkus:
            storeThreadResults[id][skuId2] = executor.submit(stockCheck1, str(id), skuId2)
    
for id, store in storeThreadResults.items():
    for skuId2, skuResult in store.items():
        storeResults[id][skuId2] = skuResult.result()

        if storeResults[id][skuId2] == "Error":
            storeErrors[skuId2].append(id)
        elif storeResults[id][skuId2] == "Not Found":
            storeErrors["notOnline"].append(id)

    if list(storeResults[id].values()).count("Not Stocked") == len(skus.keys()):
        storeNotSelling.append(id)
        del storeResults[id]

print(f"Error - {storeErrors}")
print(f"Not Selling - {storeNotSelling}")


json_output = json.dumps(storeResults)
with open(f"{skuSetName}Coles2.json", "w") as f:
    f.write(json_output)

json_output = json.dumps(storeErrors)
with open(f'{skuSetName}ColesError.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storeNotSelling)
with open(f'{skuSetName}ColesNotSelling.json', "w") as f:
    f.write(json_output)

In [20]:
#This code cell queries coles's location information API's to get Name, Address and Latitude and Longitude of a Store.

storeUrl1 = "https://apigw.coles.com.au/digital/colesweb/v1/stores/"
storeUrl2 = f"https://www.coles.com.au/_next/data/{apiVersion}/en/find-stores/coles/a/"
storeUrl3 = "https://www.coles.com.au/api/bff/locations/"
storeUrl4 = "https://www.coles.com.au/api/bff/trolley/store/"
brandId = "?brandId=2"

storeExistAPIs = ["storeExist2", "storeExist3", "storeExist1"]

def storeExist1(storeId, url = storeUrl1, brand = brandId):
    if url[len(url) -1] != "/":
        url[len(url)] = "/"
    url = f"{url}{storeId}{brand}"
    try:
        storeResult =  requests.get(url)
    except Exception as e:
        print(f"ERR - {storeId} - storeExist1 - {e}")
        return {"error": "mow"}

    store = {}
    store["id"] = storeId

    if storeResult.status_code != 200:
        store["status_code"] = storeResult.status_code
        store["error"] = storeResult.text
        print(f"WARN {storeId} - storeExist1 - {storeResult.status_code} - {storeResult.text}")

    else:
        storeJson = storeResult.json()

        store["name"] = storeJson["storeName"]
        store["address"] = f'{storeJson["address"]} {storeJson["suburb"]} {storeJson["postcode"]} {storeJson["state"]}'
        store["latitude"] = storeJson["latitude"]
        store["longitude"] = storeJson["longitude"]

    return store

def storeExist2(storeId, url = storeUrl2):
    if url[len(url) -1] != "/":
        url[len(url)] = "/"
    url = f'{url}-{storeId}.json'
    try:
        storeResult = requests.get(url, headers={"User-Agent": userAgent}, verify=False)
    except Exception as e:
        print(f"ERR - {storeId} - storeExist2 - {e}")
        return {"error": "mow"}

    store = {}
    store["id"] = storeId

    if storeResult.status_code != 200:
        store["status_code"] = storeResult.status_code
        store["error"] = storeResult.text
        print(f"WARN {storeId} - storeExist2 - {storeResult.status_code} - {storeResult.text}")

    else:
        try:
            storeJson = storeResult.json()["pageProps"]["store"]

            store["name"] = storeJson["storeName"]
            store["address"] = f'{storeJson["address"]} {storeJson["suburb"]} {storeJson["postcode"]} {storeJson["state"]}'
            store["latitude"] = storeJson["latitude"]
            store["longitude"] = storeJson["longitude"]
        except:
            print(f"ERR - {storeId} - storeExist2 - {url} - {storeResult.text}")
            return {"error": "Store Result was not a JSON"}
    return store

def storeExist3(storeId, url = storeUrl3, brand = brandId):
    if url[len(url) -1] != "/":
        url[len(url)] = "/"
    url = f'{url}{storeId}CC{storeId}{brand}'
    try:
        storeResult = requests.get(url, headers={"User-Agent": userAgent, "ocp-apim-subscription-key": subscriptionKey}, cookies={"fulfillmentStoreId": str(storeId)}, verify=False)
    except Exception as e:
        print(f"ERR - {storeId} - storeExist3 - {e}")
        return {"error": "mow"}

    store = {}
    store["id"] = storeId

    if storeResult.status_code != 200:
        store["status_code"] = storeResult.status_code
        store["error"] = storeResult.text
        print(f"WARN {storeId} - storeExist3 - {storeResult.status_code} - {storeResult.text}")

    else:
        try:
            storeJson = storeResult.json()["location"]

            store["name"] = storeJson["locationName"]
            store["address"] = f'{storeJson["address"]} {storeJson["suburb"]} {storeJson["postcode"]} {storeJson["state"]}'
            store["latitude"] = storeJson["latitude"]
            store["longitude"] = storeJson["longitude"]
        except:
            print(f"ERR - {storeId} - storeExist3 - {url} - {storeResult.text}")
            return {"error": "Store Result was not a JSON"}
    return store

# This is essentially useless. It requires a token, and returns the store info from the last store that store has used on the server.
def storeExist4(storeId, url = storeUrl4):
    if url[len(url) -1] != "/":
        url[len(url)] = "/"
    url = f'{url}{storeId}'
    try:
        storeResult = requests.get(url, headers={
                "User-Agent": userAgent,
                "ocp-apim-subscription-key": subscriptionKey,
                "x-api-version": "3",
                "Cookie": f"refreshToken={refreshToken}" + f"fulfillmentStoreId={storeId};"
            },  verify=False)
    except Exception as e:
        print(f"ERR - {storeId} - storeExist4 - {e}")
        return {"error": "mow"}

    store = {}
    store["id"] = storeId

    if storeResult.status_code != 200:
        store["status_code"] = storeResult.status_code
        store["error"] = storeResult.text
        print(f"WARN {storeId} - storeExist4 - {storeResult.status_code} -{storeResult.text}")

    else:
        try:
            storeJson = storeResult.json()["location"]

            store["name"] = storeJson["locationName"]
            store["address"] = f'{storeJson["address"]} {storeJson["suburb"]} {storeJson["postcode"]} {storeJson["state"]}'
            store["latitude"] = storeJson["latitude"]
            store["longitude"] = storeJson["longitude"]
        except:
            print(f"ERR - {storeId} - storeExist4 - {url} - {storeResult.text}")
            return {"error": "Store Result was not a JSON"}
    return store

In [ ]:
#This queries 3 APIs to get store information for each store that stocks gingernuts

storeResults = json.load(open(f'{skuSetName}Coles2.json'))
storeErrors = json.load(open(f'{skuSetName}ColesError.json'))

storeAddedInfo = {}

for api in storeExistAPIs:
    storeResultsTemp = {}
    storeAddedInfo[api] = []
    storeErrors[api] = []
    ids = [id for id, value in storeResults.items() if "address" not in value]
    
    with concurrent.futures.ThreadPoolExecutor() as executor: # optimally defined number of threads
        for id in ids:
            storeResultsTemp[id] = executor.submit(eval(api), id)
    
    for id, value in storeResultsTemp.items():
        # print(f"{id} {storeResultsTemp[id]}")
        storeResult = storeResultsTemp[id].result()

        if "error" not in storeResult:
            for key in storeResult.keys():
                storeResults[id][key] = storeResult[key]
            storeAddedInfo[api].append(id)
        else:
            print(f"Err - {storeResult}")
            storeErrors[api].append(id)

print(f"Stores With Information - {storeAddedInfo}")
print(f"Error - {storeErrors}")


json_output = json.dumps(storeResults)
with open(f'{skuSetName}Coles3.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storeErrors)
with open(f'{skuSetName}ColesError.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storeAddedInfo)
with open(f'{skuSetName}ColesAddedInfo.json', "w") as f:
    f.write(json_output)

In [ ]:
#This opens the CSV file from the ASPx webpage, which contains store information
# https://sites.coles.com.au/Sites/StoreSearch.aspx

storeResults = json.load(open(f'{skuSetName}Coles3.json'))
storeAddedInfo = json.load(open(f'{skuSetName}ColesAddedInfo.json'))
storeAddedInfo["csv"] = []

with open('LocationsData.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        id = row["Number"]
        if id in storeResults and "address" not in storeResults[id]:
            storeResults[id]["name"] = row["Name"]
            storeResults[id]["address"] = row["Address"]
            storeAddedInfo["csv"].append(id)

print(f'Stores With Information - {storeAddedInfo["csv"]}')

json_output = json.dumps(storeResults)
with open(f'{skuSetName}Coles4.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storeAddedInfo)
with open(f'{skuSetName}ColesAddedInfo.json', "w") as f:
    f.write(json_output)

In [ ]:
#This is now unneeded as the Data has been cleaned up before getting to this point. This found the locations of stores without latitude and longitude. 
from geopy.geocoders import Nominatim

storeResults = json.load(open(f'{skuSetName}Coles4.json'))

storeAddedInfo = json.load(open(f'{skuSetName}ColesAddedInfo.json'))
storeAddedInfo["osm"] = []

storeNoAddresses = {}
storeNoAddresses["osm"] = []

storeBadAddresses = {}
storeBadAddresses["osm"] = []

storeErrors = json.load(open(f'{skuSetName}ColesError.json'))
storeErrors["osm"] = []

geolocator = Nominatim(user_agent=userAgent)

for id, store in storeResults.items():
    if "latitude" not in store:
        if "address" not in store:
            storeNoAddresses["osm"].append(id)
        else:
            try:
                storeLocation = geolocator.geocode(store["address"])
                if storeLocation is None:
                    storeBadAddresses["osm"].append(store["id"])
                    print(f'ERR - {store["id"]} - {store["address"]}')
                else:
                    store["latitude"] = storeLocation.latitude
                    store["longitude"] = storeLocation.longitude
                    storeAddedInfo["osm"].append(id)
            except Exception as e:
                storeErrors["osm"].append(id)
                print(f'ERR - {store["id"]} - {e}')

print(f'Stores With Information - {storeAddedInfo["osm"]}')
print(f'Stores With No Address - {storeNoAddresses["osm"]}')
print(f'Stores With Unaccepted Address - {storeBadAddresses["osm"]}')
print(f'Stores With Errors - {storeErrors["osm"]}')

json_output = json.dumps(storeResults)
with open(f'{skuSetName}Coles5.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storeAddedInfo)
with open(f'{skuSetName}ColesAddedInfo.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storeNoAddresses)
with open(f'{skuSetName}ColesNoAddress.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storeBadAddresses)
with open(f'{skuSetName}BadAddress.json', "w") as f:
    f.write(json_output)

json_output = json.dumps(storeErrors)
with open(f'{skuSetName}ColesError.json', "w") as f:
    f.write(json_output)

In [ ]:
#This creates a set of which stores sell which skus
storeResults = json.load(open(f'{skuSetName}Coles5.json'))

skuStock = {}
skuStockInStock = {}

for sku in skus.keys():
    skuStock[sku] = set()
    skuStockInStock[sku] = set()

for id, store in storeResults.items():
    for skuId, skuResult in store.items():
        if skuResult == "In Stock":
            skuStock[str(skuId)].add(id)
            skuStockInStock[str(skuId)].add(id)
        
        elif skuResult == "Out of Stock":
            skuStock[str(skuId)].add(id)

print("In Stock in Both")
for skuId1, sku1 in skus.items():
    for skuId2, sku2 in skus.items():
        if skuId1 != skuId2:
            print(f'{sku1["display-name"]} - {sku2["display-name"]} - {skuStockInStock[skuId1].intersection(skuStockInStock[skuId2])}')

print("\nSold at Both")

for skuId1, sku1 in skus.items():
    for skuId2, sku2 in skus.items():
        if skuId1 != skuId2:
            print(f'{sku1["display-name"]} - {sku2["display-name"]} - {skuStock[skuId1].intersection(skuStock[skuId2])}')

In [ ]:
storeResults1 = json.load(open(f'savoyColes5.json'))
storeResults2 = json.load(open(f'gingernutColes5.json'))

print(set(storeResults1.keys()).difference(set(storeResults2.keys())))
print(set(storeResults2.keys()).difference(set(storeResults1.keys())))